# Jarvis Lite - Personlig Assistent (Eksamensprojekt)

Denne notebook indeholder koden til Jarvis Lite, en simpel personlig assistent udviklet som en del af eksamensprojektet.

**Funktioner:**
- Modtager tekstinput fra brugeren.
- Kan fortælle klokken.
- Kan åbne YouTube i en webbrowser.
- Kan gemme noter i en fil (`notes.txt`).
- Giver feedback via talesyntese (gTTS).

**Teknologier:**
- Python
- gTTS (Google Text-to-Speech)
- playsound
- webbrowser
- datetime
- os
- logging

--- 
Kør cellen nedenfor for at starte assistenten.

In [1]:
# KODE FRA main.py (tilpasset til notebook, med forbedringer)
import datetime
import webbrowser
import os
import logging
import time
from gtts import gTTS
from playsound import playsound
import random # Tilføjet for variation i svar

# Konfigurer simpel logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# --- Kernefunktioner ---

def get_time():
    """Returnerer den aktuelle tid som en formateret streng."""
    now = datetime.datetime.now()
    return f"Klokken er {now.strftime('%H:%M')} ⏰" # Tilføjet emoji

def open_youtube():
    """Åbner YouTube i standardbrowseren."""
    try:
        webbrowser.open("https://www.youtube.com")
        return "Jeg har åbnet YouTube for dig."
    except Exception as e:
        logging.error(f"Fejl ved åbning af YouTube: {e}")
        return "Beklager, jeg kunne ikke åbne YouTube."

def open_dr():
    """Åbner DR.dk i standardbrowseren."""
    try:
        webbrowser.open("https://www.dr.dk")
        return "Jeg har åbnet DR for dig."
    except Exception as e:
        logging.error(f"Fejl ved åbning af DR: {e}")
        return "Beklager, jeg kunne ikke åbne DR."

def save_note_command(command):
    """Udtrækker og gemmer en note fra kommandoen."""
    note_content = command.split("gem note", 1)[-1].strip()
    if note_content:
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        try:
            with open("notes.txt", "a", encoding="utf-8") as f:
                f.write(f"{timestamp}: {note_content}\n")
            logging.info(f"Note gemt: {note_content}")
            return f"Noten '{note_content}' er gemt."
        except Exception as e:
            logging.error(f"Fejl ved gemning af note: {e}")
            return "Beklager, jeg kunne ikke gemme noten."
    else:
        return "Hvad skal jeg gemme i noten?"

def say_hello():
    """Returnerer en tilfældig hilsen."""
    greetings = ["Hej med dig! Hvordan kan jeg hjælpe?", "Goddag!", "Hallo! Klar til at assistere."]
    return random.choice(greetings)

def say_identity():
    """Returnerer assistentens identitet."""
    return "Jeg er Jarvis Lite, din personlige assistent."

def say_goodbye():
    """Returnerer en afskedshilsen."""
    return "Farvel! Hav en god dag."

# --- Talesyntese ---

def speak_gtts(text, lang='da'):
    """Genererer tale med gTTS, gemmer som temp fil, afspiller og sletter."""
    if not text: # Undgå fejl hvis teksten er tom
        return False
    try:
        tts = gTTS(text=text, lang=lang, slow=False)
        temp_file = "temp_speech.mp3"
        tts.save(temp_file)
        logging.info(f"Afspiller tale: '{text}'")
        playsound(temp_file)
        time.sleep(0.1)
        os.remove(temp_file)
        return True
    except Exception as e:
        logging.error(f"Fejl under gTTS/playsound: {e}", exc_info=True)
        print(f"(gTTS Fejl - Svar): {text}")
        return False

# --- Kommando Håndtering (Med Dictionary) ---

# Definer kommandoer og deres handlinger/svar
# Nøgle: Nøgleord eller start på sætning (lowercase)
# Værdi: Funktion der skal kaldes ELLER en direkte streng som svar
command_map = {
    "hvad er klokken": get_time,
    "hvad tid er det": get_time,
    "åbn youtube": open_youtube,
    "åbn dr": open_dr,
    "gem note": save_note_command, # Bruger speciel funktion der tager hele kommandoen
    "hej": say_hello,
    "goddag": say_hello,
    "hallo": say_hello,
    "hvem er du": say_identity,
    "farvel": say_goodbye,
    "slut": say_goodbye,
    "exit": say_goodbye,
    # Tilføj flere kommandoer her...
    # "hvordan går det": "Det går strygende!",
}

def handle_text_command(command):
    """Behandler brugerens tekstkommando vha. command_map."""
    command_lower = command.lower().strip()
    response = None
    should_continue = True

    # Tjek for eksakte matches eller start-matches i dictionary
    for key, action in command_map.items():
        if command_lower.startswith(key):
            if callable(action): # Hvis værdien er en funktion
                if key == "gem note": # Særbehandling for 'gem note'
                    response = action(command) # Send hele kommandoen
                else:
                    response = action() # Kald funktionen
            else: # Hvis værdien er en streng
                response = action
            
            # Sæt should_continue til False for afslutningskommandoer
            if key in ["farvel", "slut", "exit"]:
                should_continue = False
            break # Stop med at søge når et match er fundet

    # Hvis ingen kommando blev genkendt
    if response is None:
        response = "Jeg forstod ikke kommandoen. Prøv igen."

    # Tal svaret
    speak_gtts(response)

    return should_continue

# --- Hovedprogram-løkke ---

def main_loop():
    logging.info("Jarvis Lite (Notebook version - gTTS) starter...")
    
    if not speak_gtts("Hej, jeg er Jarvis Lite. Hvad kan jeg hjælpe med?"):
         print("-> Start-hilsen printet pga. TTS fejl.")

    keep_running = True
    while keep_running:
        try:
            user_input = input("Du: ")
            if user_input:
                keep_running = handle_text_command(user_input)
        except KeyboardInterrupt:
            print("\nManuel afbrydelse modtaget.")
            speak_gtts("Lukker ned efter afbrydelse.")
            keep_running = False
        except Exception as e:
            logging.error(f"Der opstod en uventet fejl: {e}", exc_info=True)
            print(f"\nFEJL: Der opstod en uventet fejl. Se log/output for detaljer.")
            speak_gtts("Der skete en uventet fejl. Tjek output.")
            pass # Fortsæt løkken

    logging.info("Jarvis Lite hovedløkke afsluttet.")

# --- Start Assistenten ---
# Kald hovedløkken når cellen køres
main_loop()


Manuel afbrydelse modtaget.


2025-04-27 21:46:52,753 - INFO - Afspiller tale: 'Lukker ned efter afbrydelse.'
2025-04-27 21:46:55,570 - INFO - Jarvis Lite hovedløkke afsluttet.
